In [148]:
### kütüphaneleri tanımlayınız. ### 
import pandas as pd
from gensim.models import Word2Vec
import snowballstemmer
from snowballstemmer import TurkishStemmer
import re
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score,accuracy_score, confusion_matrix, ConfusionMatrixDisplay 


In [63]:
df = pd.read_csv("data/nlp.csv",index_col=0)
df

,Text
0,"Yapay Zeka Nedir, Ne İşe Yarar?\n\nYapay zeka ..."
1,🔎 Verilerinizi analiz edin: \nChatGPT'yi kulla...
2,"Öncelikle yapay zekanın ne olduğunu, #dijitala..."
3,9. Baidu Research: Çin'in önde gelen arama mot...
4,4. Microsoft AI: Microsoft'un yapay zeka tekno...
...,...
853,doğal dil işleme alanında yüksek lisans-doktor...
854,dogal dil isleme şeyleri
855,Bir veri madenciliği ve doğal dil işleme uygul...
856,"""Doğal dil işleme"" ile ilgili deneyimi olan ya..."


In [2]:
# numerik karakterlerin kaldırılması
def remove_numeric(value):
    bfr=[item for item in value if not item.isdigit()]
    bfr="".join(bfr)
    return bfr

In [3]:
# emojilerin kaldırılması
def remove_emoji(value):
    bfr=re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # simgeler ve pictographs
                           u"\U0001F680-\U0001F6FF"  # taşıtlar ve semboller
                           u"\U0001F1E0-\U0001F1FF"  # bayraklar (iOS)
                           u"\U00002702-\U000027B0"  # diğer emojiler
                           u"\U000024C2-\U0001F251"  # diğer emojiler
                           "]+",flags=re.UNICODE)
    bfr=bfr.sub(r'',value)
    return bfr

In [4]:
#noktalama işaretlerinin kaldırılması
def remove_noktalama(value):
    return re.sub(r'[^\w\b]','',value)

In [5]:
#tek karakterli ifadelerin kaldırılması
def remove_single_chracter(value):
    return re.sub(r'\b\w\b', '', value)

In [6]:
#linklerin kaldırılması 
def remove_link(value):
    return re.sub(r'http\S+|www\S+', '', value)

In [7]:
# hashtaglerin kaldırılması
def remove_hashtag(value):
    return re.sub(r'#\w+', '', value)

In [8]:
# kullanıcı adlarının kaldırılması
def remove_username(value):
    return re.sub(r'@\w+', '', value)

In [9]:
#kök indirgeme ve stop words işlemleri
def stem_word(value):
    stemmer = TurkishStemmer()
    value = value.lower()
    value = stemmer.stemWords(value.split())
    stop_words = ['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki', 'biri', 'çok',
                  'çünkü', 'da', 'de', 'defa', 'eğer', 'en', 'gibi', 'hem', 'siz', 'şu']
    value = [item for item in value if item not in stop_words]
    value = ' '.join(value)
    return value

In [10]:
# ön işlem fonksiyonlarının sırayla çağırılması
def pre_processing(value):
    return [remove_numeric(remove_emoji
                          (remove_single_chracter
                           (remove_noktalama
                            (remove_link
                             (remove_hashtag
                              (remove_username
                               (stem_word
                                (word)))))))) for word in value.split()]
    

In [12]:
# Boşlukların kaldırılması
def remove_space(value):
    return [item for item in value if item.strip()]

In [65]:
# word2vec model oluşturma ve kaydetme
    
def word2vec_create(value):
    model = Word2Vec(sentences = value.tolist(),vector_size=100,window=5,min_count=1)
    model.save("data/word2vec.model")


In [14]:
# word2vec model yükleme ve vektör çıkarma
def word2vec_analysis(value):
    model = Word2Vec.load("data/word2vec.model")
    vectors = []
    for sentence in value:
        vector = []
        for word in sentence.split():
            if word in model.wv:
                vector.append(model.wv[word])
        vectors.append(vector)
    return vectors

In [15]:
# word2vec model güncellenir.
def word2vec_update(value):
    model = Word2Vec.load("data/word2vec.model")
    model.build_vocab(value.tolist(),update=True)
    model.save("data/word2vec.model")

In [22]:
if __name__ == '__main__':
   
    # veri temizlemesi için örnek veri kümemiz okunur.
    df_1 = pd.read_csv("data/nlp.csv",index_col=0)


    ### tanımlanan df_1 içerisinde Text sütununu ön işlem fonksiyonlarından geçirerek Text_2 olarak df_1 içerisinde yeni bir sütun oluşturun. ###
    df_1['Text_2'] = df_1['Text'].apply(pre_processing)
    
    if df_1['Text_2'].isnull().values.any():
        df_1 = df_1.dropna(subset=['Text_2'])

    
    ### word2vec model oluşturma ###
    word2vec_create(df_1['Text_2'])

    #model=Word2Vec(sentences=df["Text_2"].tolist(),vector_size=100, window=5,min_count=1)
    #model.save("data/word2vec.model")

    
    # df_1 dataframe mizi artık kullanmaycağımızdan ram de yer kaplamaması adına boş bir değer ataması yapıyoruz.
    df_1 = {}


In [23]:
# sınıflandırma yapacağımız veri okunur.
df_2 = pd.read_csv("data/metin_siniflandirma.csv",index_col=0)

In [77]:
### tanımlanan df_2 içerisinde Text sütununu ön işlem fonksiyonlarından geçirerek Text_2 olarak df_2 içerisinde yeni bir sütun oluşturun. ###
df_2['Text_2'] = df_2['Text'].apply(pre_processing)
df_2

,index,Text,Label,Text_2
0,0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,kızgın,"[allah, çıldıracağım, ye, eder, çıldıracak, si..."
1,1,site ne zaman çalıştıda ürün stokları bitti di...,kızgın,"[si, ne, zama, çalıştı, ür, stok, bit, diyor, ..."
2,2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,kızgın,"[sebep, ne, ki, di, bağıracak, , , kal]"
3,3,Black friday ne amk bu yıl çıkardınız bu bokud...,kızgın,"[black, friday, ne, amk, bu, yıl, çıkar, bu, b..."
4,4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,kızgın,"[kıs, kıs, cevap, vericek, ni, yazıyor, gerize..."
...,...,...,...,...
3993,3994,Aynı şeyleri yapmaktan sıkıldım,üzgün,"[aynı, şey, yapmak, sıkıl]"
3994,3995,"Olacağı varsa oluyor ya hani, biteceği varsa d...",üzgün,"[olacak, var, oluyor, ya, hani, bitecek, var, ..."
3995,3996,Çok şey istiyorum ama sonu hep fiyasko MUTSUZUM,üzgün,"[, şey, istiyor, am, so, hep, fiyasko, mutsuz]"
3996,3997,Garip bir şekilde canım çok yanıyor moralim bozuk,üzgün,"[garip, bir, şekil, ca, , yanıyor, moral, bozuk]"


In [79]:
### df_2 içerisinde Text_2 sütununda boş liste kontrolü ###

df_2[df_2["Text_2"].str[0].isnull()]

,index,Text,Label,Text_2


In [82]:
df_2.head()

,index,Text,Label,Text_2
0,0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,kızgın,"[allah, çıldıracağım, ye, eder, çıldıracak, si..."
1,1,site ne zaman çalıştıda ürün stokları bitti di...,kızgın,"[si, ne, zama, çalıştı, ür, stok, bit, diyor, ..."
2,2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,kızgın,"[sebep, ne, ki, di, bağıracak, , , kal]"
3,3,Black friday ne amk bu yıl çıkardınız bu bokud...,kızgın,"[black, friday, ne, amk, bu, yıl, çıkar, bu, b..."
4,4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,kızgın,"[kıs, kıs, cevap, vericek, ni, yazıyor, gerize..."


In [83]:
### sınıflandırma yapacağımız df_2 içerisinde bulunan Text_2 sütun verisini word2vec verisinde güncelleyin. ###
word2vec_update(df_2['Text_2'])

In [88]:
### Text_2 sütun üzerinden word2vec adında bu modeli kullanarak yeni bir sütun yaratın

def word2vec_analysis(value):
    model = Word2Vec.load("data/word2vec.model")
    vectors = []
    for sentence_list in value:
        sentence_vectors = []
        for sentence in sentence_list:
            vector = []
            for word in sentence.split():
                if word in model.wv:
                    vector.append(model.wv[word])
            sentence_vectors.append(vector)
        vectors.append(sentence_vectors)
    return vectors

    
# Her satırda, "word2vec" sütunu, metin verilerinin vektör temsilini içerir.
df_2['word2vec'] = word2vec_analysis(df_2['Text_2'])

df_2

,index,Text,Label,Text_2,word2vec
0,0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,kızgın,"[allah, çıldıracağım, ye, eder, çıldıracak, si...","[[[0.0077830045, 0.010550979, -0.0055995965, -..."
1,1,site ne zaman çalıştıda ürün stokları bitti di...,kızgın,"[si, ne, zama, çalıştı, ür, stok, bit, diyor, ...","[[[0.0027917875, -0.003665116, -0.0024256706, ..."
2,2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,kızgın,"[sebep, ne, ki, di, bağıracak, , , kal]","[[[-0.0064300597, -0.006460773, -0.008502936, ..."
3,3,Black friday ne amk bu yıl çıkardınız bu bokud...,kızgın,"[black, friday, ne, amk, bu, yıl, çıkar, bu, b...","[[[0.00559921, 0.007904231, 0.007075621, 0.004..."
4,4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,kızgın,"[kıs, kıs, cevap, vericek, ni, yazıyor, gerize...","[[[-0.014913011, 0.048460472, 0.015135522, 0.0..."
...,...,...,...,...,...
3993,3994,Aynı şeyleri yapmaktan sıkıldım,üzgün,"[aynı, şey, yapmak, sıkıl]","[[[-0.024508374, 0.08168738, 0.024060307, 0.02..."
3994,3995,"Olacağı varsa oluyor ya hani, biteceği varsa d...",üzgün,"[olacak, var, oluyor, ya, hani, bitecek, var, ...","[[[-0.045839295, 0.1722386, 0.050299395, 0.034..."
3995,3996,Çok şey istiyorum ama sonu hep fiyasko MUTSUZUM,üzgün,"[, şey, istiyor, am, so, hep, fiyasko, mutsuz]","[[], [[-0.034319937, 0.12862776, 0.03605215, 0..."
3996,3997,Garip bir şekilde canım çok yanıyor moralim bozuk,üzgün,"[garip, bir, şekil, ca, , yanıyor, moral, bozuk]","[[[-0.00803953, 0.005275151, -0.006842514, 0.0..."


In [123]:
# 'Text_2' sütunundaki metinleri birleştirerek 'Text_3' sütununu oluşturma
df_2['Text_3'] = [' '.join(w for w in item) for item in df_2['Text_2']]

# df_2 veri çerçevesini gösterme
df_2


,index,Text,Label,Text_2,word2vec,Text_3
0,0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,kızgın,"[allah, çıldıracağım, ye, eder, çıldıracak, si...","[[[0.0077830045, 0.010550979, -0.0055995965, -...",allah çıldıracağım ye eder çıldıracak sinirimd...
1,1,site ne zaman çalıştıda ürün stokları bitti di...,kızgın,"[si, ne, zama, çalıştı, ür, stok, bit, diyor, ...","[[[0.0027917875, -0.003665116, -0.0024256706, ...",si ne zama çalıştı ür stok bit diyor mal mı oğ...
2,2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,kızgın,"[sebep, ne, ki, di, bağıracak, , , kal]","[[[-0.0064300597, -0.006460773, -0.008502936, ...",sebep ne ki di bağıracak kal
3,3,Black friday ne amk bu yıl çıkardınız bu bokud...,kızgın,"[black, friday, ne, amk, bu, yıl, çıkar, bu, b...","[[[0.00559921, 0.007904231, 0.007075621, 0.004...",black friday ne amk bu yıl çıkar bu boku mal m...
4,4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,kızgın,"[kıs, kıs, cevap, vericek, ni, yazıyor, gerize...","[[[-0.014913011, 0.048460472, 0.015135522, 0.0...",kıs kıs cevap vericek ni yazıyor gerizekal
...,...,...,...,...,...,...
3993,3994,Aynı şeyleri yapmaktan sıkıldım,üzgün,"[aynı, şey, yapmak, sıkıl]","[[[-0.024508374, 0.08168738, 0.024060307, 0.02...",aynı şey yapmak sıkıl
3994,3995,"Olacağı varsa oluyor ya hani, biteceği varsa d...",üzgün,"[olacak, var, oluyor, ya, hani, bitecek, var, ...","[[[-0.045839295, 0.1722386, 0.050299395, 0.034...",olacak var oluyor ya hani bitecek var bitsin ...
3995,3996,Çok şey istiyorum ama sonu hep fiyasko MUTSUZUM,üzgün,"[, şey, istiyor, am, so, hep, fiyasko, mutsuz]","[[], [[-0.034319937, 0.12862776, 0.03605215, 0...",şey istiyor am so hep fiyasko mutsuz
3996,3997,Garip bir şekilde canım çok yanıyor moralim bozuk,üzgün,"[garip, bir, şekil, ca, , yanıyor, moral, bozuk]","[[[-0.00803953, 0.005275151, -0.006842514, 0.0...",garip bir şekil ca yanıyor moral bozuk


In [127]:
vectorizer = CountVectorizer()
X=vectorizer.fit_transform(df_2["Text_3"].tolist())
X

<3998x7408 sparse matrix of type '<class 'numpy.int64'>'
	with 34264 stored elements in Compressed Sparse Row format>

In [134]:
vectorizer = TfidfVectorizer()
X=vectorizer.fit_transform(df_2["Text_3"].tolist())
X

<3998x7408 sparse matrix of type '<class 'numpy.float64'>'
	with 34264 stored elements in Compressed Sparse Row format>

In [142]:
df_2=pd.read_csv("data/metin_siniflandirma.csv",index_col=0)  #dosyayı okuduk
df_2["Text_2"]=df_2["Text"].apply(pre_processing)  
df_2["Text_2"]=df_2["Text_2"].apply(remove_space)     #boşlukları kaldırma fonksiyonu
df_2["Text_3"]=[' '.join(wrd for wrd in x)for x in df_2["Text_2"]]   #virgülle ayrılmış halini birleştirme

df_2_index=df[df_2["Text_2"].str[0].isnull()].index    #boş listeleri silme
df_2=df_2.drop(df_2_index)
df_2=df_2.reset_index()
del df_2["index"]

df_2.head()

C:\Users\seyda\AppData\Local\Temp\ipykernel_8728\59570125.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_2_index=df[df_2["Text_2"].str[0].isnull()].index    #boş listeleri silme


,Text,Label,Text_2,Text_3
0,Allah'ım çıldıracağım. Yemin ederim çıldıracağ...,kızgın,"[allah, çıldıracağım, ye, eder, çıldıracak, si...",allah çıldıracağım ye eder çıldıracak sinirimd...
1,site ne zaman çalıştıda ürün stokları bitti di...,kızgın,"[si, ne, zama, çalıştı, ür, stok, bit, diyor, ...",si ne zama çalıştı ür stok bit diyor mal mı oğ...
2,Sebebi neydi ki diye bağıracağım şimdi az kaldı,kızgın,"[sebep, ne, ki, di, bağıracak, kal]",sebep ne ki di bağıracak kal
3,Black friday ne amk bu yıl çıkardınız bu bokud...,kızgın,"[black, friday, ne, amk, bu, yıl, çıkar, bu, b...",black friday ne amk bu yıl çıkar bu boku mal m...
4,kısa kısa cevaplar vericeksen niye yazıyorsun ...,kızgın,"[kıs, kıs, cevap, vericek, ni, yazıyor, gerize...",kıs kıs cevap vericek ni yazıyor gerizekal


In [158]:
x_train, x_test, y_train, y_test = train_test_split(df_2["Text_3"].tolist(),
                                                                df_2["Label"].tolist(), 
                                                                test_size=0.2, 
                                                                random_state=42)

In [159]:
svm=Pipeline([('vect',TfidfVectorizer()),('svm',LinearSVC())])

svm.fit(x_train,y_train)
y_pred_class=svm.predict(x_test)

In [160]:
print("svm accuracy score :", accuracy_score(y_test, y_pred_class))

svm accuracy score : 0.9775


In [132]:
### word2vec sütunumuzu train test olarak bölün ###

X_train, X_test, y_train, y_test = train_test_split(df_2['word2vec'], df_2['Label'], test_size=0.2, random_state=42)


In [163]:
### accuracy ve f1 score çıktısını print ile gösterin. ###
accuracy = accuracy_score(y_test, y_pred_class)
f1 = f1_score(y_test, y_pred_class, average='weighted')
print("Accuracy:", accuracy)
print("F1 Score:", f1)


Accuracy: 0.9775
F1 Score: 0.9775152698970822
